# if constexpr 编译期分支 - 实践篇

本 notebook 通过实际代码帮助你理解 C++17 的 `if constexpr` 特性。

**学习目标：**
- 理解 if constexpr 与普通 if 的区别
- 掌握编译期分支的使用场景
- 学会使用 if constexpr 实现类型分发
- 理解 FlashAttention 中 BOOL_SWITCH 的原理

## 环境准备

我们使用 `%%writefile` 将代码写入文件，然后使用 g++ 编译执行。

**注意：** 需要 C++17 或更高版本的编译器支持。

In [ ]:
# 检查编译器版本
!g++ --version

## 1. if constexpr 基础

首先理解 if constexpr 如何解决普通 if 无法解决的问题。

In [ ]:
%%writefile if_constexpr_basic.cpp
#include <iostream>
#include <type_traits>
#include <string>

// 使用 if constexpr 解决类型不同操作的问题
template <typename T>
void process(T value) {
    std::cout << "处理值: ";
    
    if constexpr (std::is_integral_v<T>) {
        // 只有当 T 是整数类型时，这段代码才会被编译
        std::cout << "整数类型，取模结果: " << (value % 2) << std::endl;
    } else if constexpr (std::is_floating_point_v<T>) {
        // 只有当 T 是浮点类型时，这段代码才会被编译
        std::cout << "浮点类型，四舍五入: " << static_cast<int>(value + 0.5) << std::endl;
    } else if constexpr (std::is_same_v<T, std::string>) {
        // 只有当 T 是 string 时，这段代码才会被编译
        std::cout << "字符串类型，长度: " << value.length() << std::endl;
    } else {
        std::cout << "其他类型" << std::endl;
    }
}

int main() {
    std::cout << "===== if constexpr 基础演示 =====\n" << std::endl;
    
    process(42);           // 整数
    process(3.14159);      // 浮点数
    process(std::string("Hello"));  // 字符串
    
    std::cout << "\n注意: 每种类型只编译对应的分支代码！" << std::endl;
    
    return 0;
}

In [ ]:
!g++ -std=c++17 -o if_constexpr_basic if_constexpr_basic.cpp && ./if_constexpr_basic

## 2. 指针解引用示例

一个经典的用例：根据类型是否为指针，选择解引用或直接返回。

In [ ]:
%%writefile pointer_deref.cpp
#include <iostream>
#include <type_traits>

// 获取值：如果是指针则解引用，否则直接返回
template <typename T>
auto get_value(T t) {
    if constexpr (std::is_pointer_v<T>) {
        std::cout << "  (解引用指针) ";
        return *t;  // T 不是指针时，这行代码不会被编译
    } else {
        std::cout << "  (直接返回) ";
        return t;
    }
}

// 递归解引用：处理多级指针
template <typename T>
auto deep_get_value(T t) {
    if constexpr (std::is_pointer_v<T>) {
        return deep_get_value(*t);  // 递归解引用
    } else {
        return t;
    }
}

int main() {
    std::cout << "===== 指针处理示例 =====\n" << std::endl;
    
    int x = 42;
    int* px = &x;
    int** ppx = &px;
    
    std::cout << "原始值 x = 42\n" << std::endl;
    
    std::cout << "get_value(x) = ";
    std::cout << get_value(x) << std::endl;
    
    std::cout << "get_value(px) = ";
    std::cout << get_value(px) << std::endl;
    
    std::cout << "\n递归解引用:" << std::endl;
    std::cout << "deep_get_value(x) = " << deep_get_value(x) << std::endl;
    std::cout << "deep_get_value(px) = " << deep_get_value(px) << std::endl;
    std::cout << "deep_get_value(ppx) = " << deep_get_value(ppx) << std::endl;
    
    return 0;
}

In [ ]:
!g++ -std=c++17 -o pointer_deref pointer_deref.cpp && ./pointer_deref

## 3. 变参模板递归终止

`if constexpr` 可以优雅地处理变参模板的递归终止。

In [ ]:
%%writefile variadic_print.cpp
#include <iostream>
#include <string>

// 传统方式：需要两个重载函数
namespace traditional {
    // 递归终止
    void print() {
        std::cout << std::endl;
    }
    
    // 递归展开
    template <typename T, typename... Args>
    void print(T first, Args... rest) {
        std::cout << first;
        if (sizeof...(rest) > 0) std::cout << ", ";
        print(rest...);
    }
}

// 使用 if constexpr：单个函数搞定
namespace modern {
    template <typename T, typename... Args>
    void print(T first, Args... rest) {
        std::cout << first;
        
        if constexpr (sizeof...(rest) > 0) {
            std::cout << ", ";
            print(rest...);  // 递归调用
        } else {
            std::cout << std::endl;  // 终止
        }
    }
}

// 求和函数
template <typename T, typename... Args>
auto sum(T first, Args... rest) {
    if constexpr (sizeof...(rest) == 0) {
        return first;
    } else {
        return first + sum(rest...);
    }
}

int main() {
    std::cout << "===== 变参模板与 if constexpr =====\n" << std::endl;
    
    std::cout << "传统方式 print: ";
    traditional::print(1, 2.5, "hello", 'a');
    
    std::cout << "现代方式 print: ";
    modern::print(1, 2.5, "hello", 'a');
    
    std::cout << "\nsum(1, 2, 3, 4, 5) = " << sum(1, 2, 3, 4, 5) << std::endl;
    std::cout << "sum(1.5, 2.5, 3.0) = " << sum(1.5, 2.5, 3.0) << std::endl;
    
    return 0;
}

In [ ]:
!g++ -std=c++17 -o variadic_print variadic_print.cpp && ./variadic_print

## 4. 模拟 FlashAttention 的 BOOL_SWITCH

FlashAttention 使用 BOOL_SWITCH 宏将运行时布尔值转换为编译期常量。这是理解 FlashAttention 代码的关键。

In [ ]:
%%writefile bool_switch.cpp
#include <iostream>
#include <type_traits>

// 模拟 FlashAttention 的 BOOL_SWITCH 宏
#define BOOL_SWITCH(COND, CONST_NAME, ...)      \
    [&] {                                        \
        if (COND) {                              \
            constexpr bool CONST_NAME = true;    \
            return __VA_ARGS__();                \
        } else {                                 \
            constexpr bool CONST_NAME = false;   \
            return __VA_ARGS__();                \
        }                                        \
    }()

// 模拟不同配置的 kernel
template <bool IsCausal>
void attention_kernel() {
    std::cout << "  运行 attention_kernel<" 
              << (IsCausal ? "true" : "false") << ">" << std::endl;
    
    if constexpr (IsCausal) {
        std::cout << "  -> 应用因果掩码" << std::endl;
    } else {
        std::cout << "  -> 无掩码（全注意力）" << std::endl;
    }
}

// 使用 BOOL_SWITCH 的调度函数
void run_attention(bool is_causal) {
    std::cout << "\n调用 run_attention(is_causal=" 
              << (is_causal ? "true" : "false") << ")" << std::endl;
    
    BOOL_SWITCH(is_causal, IsCausal, [&] {
        // IsCausal 在这里是编译期常量！
        attention_kernel<IsCausal>();
    });
}

// 多层嵌套示例（类似 FlashAttention 的实际用法）
template <bool IsCausal, bool IsLocal>
void full_kernel() {
    std::cout << "  kernel<IsCausal=" << IsCausal 
              << ", IsLocal=" << IsLocal << ">" << std::endl;
}

void dispatch(bool is_causal, bool is_local) {
    std::cout << "\n调度: is_causal=" << is_causal 
              << ", is_local=" << is_local << std::endl;
    
    BOOL_SWITCH(is_causal, IsCausal, [&] {
        BOOL_SWITCH(is_local, IsLocal, [&] {
            full_kernel<IsCausal, IsLocal>();
        });
    });
}

int main() {
    std::cout << "===== BOOL_SWITCH 宏演示 =====" << std::endl;
    
    std::cout << "\n--- 单层 BOOL_SWITCH ---" << std::endl;
    run_attention(true);
    run_attention(false);
    
    std::cout << "\n--- 嵌套 BOOL_SWITCH ---" << std::endl;
    dispatch(true, true);
    dispatch(true, false);
    dispatch(false, true);
    dispatch(false, false);
    
    std::cout << "\n关键点:" << std::endl;
    std::cout << "- 运行时的 bool 值通过宏转换为编译期常量" << std::endl;
    std::cout << "- 每种组合生成独立的模板实例" << std::endl;
    std::cout << "- 编译器可以针对每种配置优化" << std::endl;
    
    return 0;
}

In [ ]:
!g++ -std=c++17 -o bool_switch bool_switch.cpp && ./bool_switch

## 5. if constexpr vs SFINAE 对比

两种方式都可以实现编译期条件选择，但 if constexpr 更加简洁。

In [ ]:
%%writefile compare_sfinae.cpp
#include <iostream>
#include <type_traits>

// ==================== SFINAE 方式 ====================
namespace sfinae_way {
    // 针对整数类型
    template <typename T>
    std::enable_if_t<std::is_integral_v<T>, void>
    process(T value) {
        std::cout << "SFINAE: 整数 " << value << ", 取模结果: " << (value % 2) << std::endl;
    }
    
    // 针对浮点类型
    template <typename T>
    std::enable_if_t<std::is_floating_point_v<T>, void>
    process(T value) {
        std::cout << "SFINAE: 浮点 " << value << ", 四舍五入: " << static_cast<int>(value + 0.5) << std::endl;
    }
}

// ==================== if constexpr 方式 ====================
namespace constexpr_way {
    // 单个函数处理所有情况
    template <typename T>
    void process(T value) {
        if constexpr (std::is_integral_v<T>) {
            std::cout << "if constexpr: 整数 " << value << ", 取模结果: " << (value % 2) << std::endl;
        } else if constexpr (std::is_floating_point_v<T>) {
            std::cout << "if constexpr: 浮点 " << value << ", 四舍五入: " << static_cast<int>(value + 0.5) << std::endl;
        } else {
            std::cout << "if constexpr: 其他类型" << std::endl;
        }
    }
}

int main() {
    std::cout << "===== SFINAE vs if constexpr =====\n" << std::endl;
    
    std::cout << "处理整数 42:" << std::endl;
    sfinae_way::process(42);
    constexpr_way::process(42);
    
    std::cout << "\n处理浮点 3.7:" << std::endl;
    sfinae_way::process(3.7);
    constexpr_way::process(3.7);
    
    std::cout << "\n结论:" << std::endl;
    std::cout << "- SFINAE: 需要多个重载函数" << std::endl;
    std::cout << "- if constexpr: 一个函数搞定，代码更清晰" << std::endl;
    
    return 0;
}

In [ ]:
!g++ -std=c++17 -o compare_sfinae compare_sfinae.cpp && ./compare_sfinae

## 6. 清理临时文件

In [ ]:
!rm -f if_constexpr_basic pointer_deref variadic_print bool_switch compare_sfinae
!rm -f *.cpp
print("临时文件已清理")

## 总结

通过本 notebook，你应该理解了：

### if constexpr 的核心特点

1. **编译期分支消除**：未选中的分支代码不参与编译
2. **类型安全**：可以在不同分支使用不兼容的操作
3. **零运行时开销**：分支在编译期就已确定

### 主要应用场景

| 场景 | 说明 |
|------|------|
| 类型分发 | 根据类型选择不同处理逻辑 |
| 递归终止 | 变参模板的优雅终止 |
| 算法选择 | 编译期选择最优算法 |
| 功能开关 | 类似 FlashAttention 的 BOOL_SWITCH |

### FlashAttention 中的应用

- `BOOL_SWITCH` 宏将运行时布尔值转换为编译期常量
- 每种配置组合（causal/local/dropout等）生成独立的 kernel
- 编译器可以针对每种配置进行优化，消除无用分支

## 练习

1. 实现一个 `TYPE_SWITCH` 宏，支持多种类型的分发
2. 使用 if constexpr 实现编译期类型名获取函数
3. 实现一个泛型序列化函数，根据类型选择不同的序列化方式